<a href="https://colab.research.google.com/github/laibachaudry49/agentic-Ai-project3/blob/main/function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain_community langchain_google_genai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [ ]:
from google.colab import userdata
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash',verbose=True)

llm.invoke("Hi")

AIMessage(content='Hi there! How can I help you today?\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-92eef083-b175-49aa-adb0-05e777a5722d-0', usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})

In [ ]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

llm_with_tools = llm.bind_tools([multiply])

In [ ]:
tools_list=[multiply]
model_with_tools = llm.bind_tools(tools_list)

In [ ]:
result = llm_with_tools.invoke("Hey how are you!")
result

AIMessage(content="I'm doing well, thank you for asking! How are you today?\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-821e8bc5-b803-4b93-8d28-66de3e444481-0', usage_metadata={'input_tokens': 68, 'output_tokens': 17, 'total_tokens': 85, 'input_token_details': {'cache_read': 0}})

In [ ]:
result = llm_with_tools.invoke("What is 2 multiplied by 3?")
result



AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.0, "b": 3.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-432f85cc-f0bf-4c25-9da9-1b17c663c42d-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2.0, 'b': 3.0}, 'id': '4b4d136a-b285-4bfe-8ce2-d2fe318888d9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 72, 'output_tokens': 3, 'total_tokens': 75, 'input_token_details': {'cache_read': 0}})

In [ ]:
from langchain.tools import Tool

# Example tool for addition
def add_numbers(a, b):
    return a + b

math_tool = Tool(
    name="AdditionTool",
    func=lambda x: f"The result is: {add_numbers(*map(int, x.split()))}",
    description="Adds two numbers. Input should be two integers separated by a space."
)


In [ ]:

from langchain.agents import initialize_agent, AgentType

# List of tools
tools = [math_tool]

# Initialize the agent
agent = initialize_agent(
    tools=tools,                        # Provide the tools
    llm=llm,                            # LLM for fallback
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Agent type
    verbose=True                        # Enable debugging output
)

<ipython-input-9-f11f73b77070>:7: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [ ]:
# Query the agent
response1 = agent.run("What is 10 + 20?")  # Uses the tool
print(response1)



<ipython-input-10-4661abfb96ad>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response1 = agent.run("What is 10 + 20?")  # Uses the tool




> Entering new AgentExecutor chain...
Thought: I need to add 10 and 20.  I can use the AdditionTool for this.
Action: AdditionTool
Action Input: 10 20
Observation: The result is: 30
Thought:I now know the final answer
Final Answer: 30


> Finished chain.
30
